# 🚀 PiEdge EduKit - Snabbkörning & Sanity Check

**Mål**: Kör hela lektionen snabbt och se att allt fungerar.

Detta notebook kör samma steg som `run_lesson.sh`, men som klickbara celler. Perfekt för att:
- Komma igång snabbt
- Se helheten innan vi går in på detaljer
- Verifiera att din miljö fungerar

> **💡 Tips**: Kör cellerna i ordning. Om något går fel, kolla att du har Python 3.12 och att `.venv` är aktiverat.


## 1️⃣ Setup & Verifiering

Först kontrollerar vi att miljön är korrekt:


In [ ]:
# Kontrollera Python-version
import sys
print(f"Python version: {sys.version}")
assert sys.version_info[:2] == (3, 12), f"Python 3.12 krävs, du har {sys.version_info[:2]}"
print("✅ Python 3.12 OK")


In [ ]:
# Kontrollera att paketet är installerat
try:
    import piedge_edukit
    print("✅ PiEdge EduKit package OK")
except ImportError:
    print("❌ Package inte installerat. Kör: pip install -e .")
    raise


## 2️⃣ Träning & ONNX Export

Tränar en liten modell med FakeData och exporterar till ONNX:


In [ ]:
# Träna modell (snabb körning för demo)
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 256 --output-dir ./models


In [ ]:
# Kontrollera att modellen skapades
import os
if os.path.exists("./models/model.onnx"):
    size_mb = os.path.getsize("./models/model.onnx") / (1024*1024)
    print(f"✅ ONNX-modell skapad: {size_mb:.1f} MB")
else:
    print("❌ ONNX-modell saknas")


## 3️⃣ Latensbenchmark

Mäter hur snabb modellen är på CPU:


In [ ]:
# Kör benchmark (snabb körning)
!python -m piedge_edukit.benchmark --fakedata --model-path ./models/model.onnx --warmup 1 --runs 3 --providers CPUExecutionProvider


In [ ]:
# Visa benchmark-resultat
if os.path.exists("./reports/latency_summary.txt"):
    with open("./reports/latency_summary.txt", "r") as f:
        print("📊 Benchmark-resultat:")
        print(f.read())
else:
    print("❌ Benchmark-rapport saknas")


## 4️⃣ Kvantisering (INT8)

Komprimerar modellen för snabbare inference:


In [ ]:
# Kör kvantisering
!python -m piedge_edukit.quantization --fakedata --model-path ./models/model.onnx --calib-size 16


In [ ]:
# Visa kvantiseringsresultat
if os.path.exists("./reports/quantization_summary.txt"):
    with open("./reports/quantization_summary.txt", "r") as f:
        print("⚡ Kvantiseringsresultat:")
        print(f.read())
else:
    print("❌ Kvantiseringsrapport saknas")


## 5️⃣ Utvärdering & Verifiering

Testar modellen och genererar kvitto:


In [ ]:
# Kör utvärdering
!python scripts/evaluate_onnx.py --model ./models/model.onnx --fakedata --limit 16


In [ ]:
# Kör verifiering och generera kvitto
!python verify.py


In [ ]:
# Visa kvitto
import json
if os.path.exists("./progress/receipt.json"):
    with open("./progress/receipt.json", "r") as f:
        receipt = json.load(f)
    print("📋 Verifieringskvitto:")
    print(f"Status: {'✅ PASS' if receipt['pass'] else '❌ FAIL'}")
    print(f"Timestamp: {receipt['timestamp']}")
    print("\nKontroller:")
    for check in receipt['checks']:
        status = "✅" if check['ok'] else "❌"
        print(f"  {status} {check['name']}: {check['reason']}")
else:
    print("❌ Kvitto saknas")


## 🎉 Klar!

Du har nu kört hela PiEdge EduKit-lektionen! 

**Nästa steg**: Gå till `01_training_and_export.ipynb` för att förstå vad som hände under träningen.

**Genererade filer**:
- `models/model.onnx` - Tränad modell
- `reports/` - Benchmark och kvantiseringsrapporter
- `progress/receipt.json` - Verifieringskvitto
